In [2]:
import numpy as np

In [2]:
from sklearn.metrics.pairwise import euclidean_distances

In [3]:
positions = np.arange(10).reshape(-1,2)

In [4]:
positions

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [5]:
dist_matrix = euclidean_distances(positions)
dist_matrix

array([[ 0.        ,  2.82842712,  5.65685425,  8.48528137, 11.3137085 ],
       [ 2.82842712,  0.        ,  2.82842712,  5.65685425,  8.48528137],
       [ 5.65685425,  2.82842712,  0.        ,  2.82842712,  5.65685425],
       [ 8.48528137,  5.65685425,  2.82842712,  0.        ,  2.82842712],
       [11.3137085 ,  8.48528137,  5.65685425,  2.82842712,  0.        ]])

In [6]:
dist_matrix[dist_matrix<5] = 0

In [7]:
dist_matrix

array([[ 0.        ,  0.        ,  5.65685425,  8.48528137, 11.3137085 ],
       [ 0.        ,  0.        ,  0.        ,  5.65685425,  8.48528137],
       [ 5.65685425,  0.        ,  0.        ,  0.        ,  5.65685425],
       [ 8.48528137,  5.65685425,  0.        ,  0.        ,  0.        ],
       [11.3137085 ,  8.48528137,  5.65685425,  0.        ,  0.        ]])

In [1]:
from spektral.layers import GraphConv
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.backend import gather
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2


class GraphicEncoder():
    def __init__(self, F):
        self.model = self.build_model(F)

    def build_model(self,F):
        X_in = Input(shape=(F, ), name='X_in',batch_size=1)
        A_in = Input(shape=(None,), name='A_in',batch_size=1)
        RL_indice = Input(shape=(1,),name='RL_indice',dtype='int32',batch_size=1)

        x = GraphConv(32, activation='relu',name='gcn1')([X_in, A_in])
        x = GraphConv(32, activation='relu',name='gcn2')([x, A_in])
        x = Lambda(lambda x: gather(x,RL_indice),name='slice')(x)

        model = Model(inputs = [X_in,A_in,RL_indice], outputs=x)
        print(model.summary())
        return model

In [ ]:
num_lanes = 3
num_unique_intentions = len(set(intention_dic.values()))
feature_size = 3 + num_lanes + num_unique_intentions
rl_model = GraphicEncoder(feature_size)
